<a href="https://colab.research.google.com/github/rayeeed/UFP_Project/blob/main/googlePOI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unicodecsv

  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10765 sha256=411f19783b50f4a31f735cf08ebe84bf046a865d9119ff463e646c4ecd78dd4f
  Stored in directory: /root/.cache/pip/wheels/1a/f4/8a/a5024fb77b32ed369e5c409081e5f00fbe3b92fdad653f6e69
Successfully built unicodecsv


In [ ]:
import requests

In [ ]:
import unicodecsv as csv
import pandas as pd
import json
import time
import numbers

In [ ]:
# inputs parameters
lat_lon_file = "/content/output_count_289.csv" # the file that contain centroids points
google_places_types = "/content/google_places_types.csv" # the file that contain all google places types
api_key = "" # insert your google api key

search_radius = 1000 #METER
counter = 0
querylimit = 1000

xy = pd.read_csv(lat_lon_file)
cleaned_xy = xy.loc[(xy['latitude']>=1) & (xy['longitude']<0),:]

google_types = pd.read_csv(google_places_types, header = None)

In [ ]:
google_types.head()

,0
0,laundry
1,painter
2,car repair
3,gas station
4,restaurants


In [ ]:
def get_attribute_value(place, attribute):
    if attribute in place:
        return place[attribute]
    else:
        return "N/A"

def clean_json(place, objectid):
    place_details = [objectid,place['place_id'],place['name'], place['reference'], place['geometry']['location']['lat'],
                     place['geometry']['location']['lng'], get_attribute_value(place,"vicinity"),
                     get_attribute_value(place,"price_level"), get_attribute_value(place,"rating")]
    return place_details

In [ ]:
def find_google_places(lat, lon, search_radius, search_type, api_key, objectid):
    global token_list
    global counter
    #construct url for python to open
    location = str(lat)+","+str(lon)
    search_radius = search_radius
    if counter < querylimit:
        myurl = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=%s&radius=%s&types=%s&key=%s' % (location, search_radius, search_type, api_key)
        response = urllib2.urlopen(myurl)
        counter += 1
        html = response.read()
        places = json.loads(html)
        response.close()

        places_details = []
        next_page_token = ""

        if places['status'] == 'OK':
            for place in places["results"]:
                place_details = clean_json(place,objectid)
                places_details.append(place_details)

        if "next_page_token" in places:
            next_page_token = places['next_page_token']
            token_list.append([objectid, next_page_token])
    else:
        return "over limit"


    return places_details # in the format of json!

In [ ]:
def find_google_places_nextpage(token, api_key, objectid):
    global token_list
    global counter
    if counter < querylimit:
        myurl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken=%s&key=%s"%(token,api_key)
        result = urllib2.urlopen(myurl)
        counter += 1
        content = result.read()
        places = json.loads(content)
        places_details = []
        next_page_token = ""

        if places['status'] == 'OK':
            for place in places["results"]:
                place_details = clean_json(place,objectid)
                places_details.append(place_details)

        if "next_page_token" in places:
            next_page_token = places['next_page_token']
            token_list.append([objectid, next_page_token])
        return places_details
    else:
        return "over limit"

In [ ]:
from io import BytesIO
import codecs

In [ ]:
class UnicodeWriter:
    """
    A CSV writer which will write rows to CSV file "f",
    which is encoded in the given encoding.
    """

    def __init__(self, f, dialect=csv.excel, encoding="utf-8", **kwds):
        # Redirect output to a queue
        self.queue = BytesIO()
        self.writer = csv.writer(self.queue, dialect=dialect, **kwds)
        self.stream = f
        self.encoder = codecs.getincrementalencoder(encoding)()

    def writerow(self, row):
        self.writer.writerow([s.encode("utf-8") for s in row])
        # Fetch UTF-8 output from the queue ...
        data = self.queue.getvalue()
        data = data.decode("utf-8")
        # ... and reencode it into the target encoding
        data = self.encoder.encode(data)
        # write to the target stream
        self.stream.write(data)
        # empty queue
        self.queue.truncate(0)

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [ ]:
for i in range(0,len(google_types)):  # for each google place type
    search_type = google_types.iloc[i, 0]
    cleaned_xy[search_type] = 0
    token_list = []

    output_file = "/content/ places_%s.csv" % search_type 
    with open(output_file,"wb") as outputfile:
        csvwriter = UnicodeWriter(outputfile)
        csvwriter.writerow(['OBJECTID','place_id','name', 'reference', 'lat','lng','vicinity',   
                         "price_level","rating"]) #restaurant, gas station, market, bus stop
        print ("start searching for %s" %search_type)
        for index, row in cleaned_xy.iterrows():
            print (index, counter)
            lat = row["latitude"]
            lon = row["longitude"]
            objectid = index
            try:
                places = find_google_places(lat, lon, search_radius, search_type, api_key, objectid)
                if places == "over limit":
                    print ("over limit",search_type, objectid)
                    break
            except:
                time.sleep(2)
                places = find_google_places(lat, lon, search_radius, search_type, api_key, objectid)

            cleaned_xy.loc[index, search_type] = len(places)
            # write places to csv
            res = []
            for row in places:
                res.append([str(s) if isinstance(s, numbers.Number) else s for s in row])
            csvwriter.writerows(res)

        time.sleep(2)
        while len(token_list)>0:
            [objectid, token] = token_list.pop(0)
            print (objectid, len(token_list))
            try:
                places = find_google_places_nextpage(token, api_key, objectid)
                if places == "over limit":
                    print ("over limit",search_type, objectid)
                    break
            except:
                time.sleep(2)
                places = find_google_places_nextpage(token, api_key, objectid)
            cleaned_xy.loc[objectid, search_type] += len(places)
            res = []
            for row in places:
                res.append([str(s) if isinstance(s, numbers.Number) else s for s in row])
            csvwriter.writerows(res)

    cleaned_xy.to_csv(lat_lon_file)
    print ("%s.csv saved" %search_type)
    print ('-'*80)

start searching for laundry
laundry.csv saved
--------------------------------------------------------------------------------
start searching for painter
painter.csv saved
--------------------------------------------------------------------------------
start searching for car repair
car repair.csv saved
--------------------------------------------------------------------------------
start searching for gas station
gas station.csv saved
--------------------------------------------------------------------------------
start searching for restaurants
restaurants.csv saved
--------------------------------------------------------------------------------
